# Data Analytics Spring 2023 &mdash; Exercises 2

### Onni Roivas (last modified: Fri Jan 27 at 09:05 - peer review polish)

- Five problems + round 1 peer review
- Round 1 peer review deadline: **Mon Jan 30 at 14:00**
- Problems deadline: **Mon Feb 6 at 14:00**
- Theme: data wrangling with **pandas** (please use pandas in each problem)
- **Make a copy of the original notebook** (right click & duplicate) and add your answers (new cells) there
- Remember: **no autosave** (so keep hitting that ctrl-s / cmd-s button)
- Please make both your code and your notebook readable
- When you are done, run the handin code cell at the end of this notebook
- The original notebook may change after publication, but the
  changes should be minor
- Keep your originals up to date by running the code cell below:

In [46]:
import os
os.system("/usr/bin/bash /home/varpha/data_analytics/bin/config.sh");


Configuring...
Done.



## Round 1 peer review

**Deadline: Mon Jan 30 at 14:00**

In case you handed in your round 01 exercises, your folder **private/exrc_01/peer_review** should contain an anonymous round 01 solutions notebook of another student. In addition, some model solutions for round 01 are to be found in the folder **public/model_solutions**.

Write a few paragraphs of text (plain or markdown) into your favourite editor and submit by running the code cell below. Please address the following issues:
 
- Are the solutions okay? Can you understand / run the code?<br/>
  (as opposed to some wishful brainless copy-pasting done in a hurry)
- What do you think about the solutions?
- How many points out of 5 do they deserve as a whole?
- How many points would you give to yourself and why?
- Any feedback or comments to Harri?

Harri will read and grade your reviews as follows:
- nonexistent or nearly so = 0p
- something written = 1-2p
- well written 3p.

Please ignore the **review_by_AB0410.txt** file in your peer_review directory (see [this Teams message](https://teams.microsoft.com/l/message/19:wpZLQbHG775XujahAR4ScrnPZGiByh6yuWgewJCHXVQ1@thread.tacv2/1674749772631?tenantId=6e9eaaf0-3ff7-4de9-8cd4-1ffbd45951b9&groupId=ba0bbd60-8a1b-42b8-84ff-6de6a3c45ab6&parentMessageId=1674749772631&teamName=Data%20Analytics%20YAMK%2C%20Spring%202023&channelName=General&createdTime=1674749772631&allowXTenantAccess=false)).

Instead, **please run the code cell below**. It asks you to copy-paste your review text to a field that will become visible once you run the cell.


In [ ]:
import sys
sys.path.append("/home/varpha/data_analytics/lib")
from handin import submit_peer_review
submit_peer_review()

You may double check your review submission by running (copy-paste, press enter) the following in the terminal:

> **cat /home/varpha/data_analytics/private/AB0410/exrc_01/peer_review/review_by_AB0410.txt**

## Problem 1. Profiles

The file **private/exrc_02/data/AB0410_prob01_profiles.csv** contains some user profiles.
Read the csv into a pandas DataFrame and reorganize it as follows:

a) Separate the name and address columns so that there are separate columns for
- first name
- last name
- street address
- state
- postal code.

Keep also the ssn, username, sex, mail and birthdate columns. Drop all the other columns.

b) Print all entries where the last name begins with the letter S, sorted by:
- sex (ladies first)
- state (alphabetically)
- age (youngest first).

(Print the entries three times, in three different ways.)

In [ ]:
import getpass as gt
import pandas as pd

user = gt.getuser().upper()
df = pd.read_csv(f"./data/{user}_prob01_profiles.csv")

df["first_name"] = df["name"].str.split().str[0]
df["first_name"] = df["name"].str.extract(r"(?:Mr\.|Mrs\.|Dr\.)?\s*([A-Z][a-z]+)") #get rid of these mr. or mrs. prefixes
df["last_name"] = df["name"].str.split().str[-1]
df.loc[df["last_name"].isin(["Jr."]), "last_name"] = df["name"].str.split().str[-2] + " " + df["name"].str.split().str[-1] #finally from name to last_name and first_name

#split the address column into separate columns
df[["street_address", "state", "postal_code"]] = df["address"].str.extract(r"(.*?),\s*([A-Z]{2})\s*(\d{5}(?:-\d{4})?)", expand=True)

#drop some columns
df = df.drop(columns = ["name", "address", "website", "residence", "mail", "blood_group", "job", "company"])

#sorting the output
df = df.sort_values(by=["sex", "state", "birthdate"])

df #final output missing some states and street addresses

## Problem 2. Weather (part 1/2)

The file **private/exrc_02/data/AB0410_prob02_weather.csv** contains hourly weather observations from Helsinki during one month, downloaded from [fmi.fi](https://en.ilmatieteenlaitos.fi/open-data-manual-fmi-wfs-services) (not recommended).

First, please do some data cleaning and reorganizing as you find suitable. Then, please answer the following questions:

a) How many percentages of the tmax observations are at most one standard deviation away from the total average of (tmax - tmin)?

b) Find the top-5 timestamps for the difference between tmax and tmin. For the found rows, print out the following information: timestamp, max temperature, min temperature and temperature difference.


In [ ]:
import getpass as gt
import pandas as pd

user = gt.getuser().upper()
df = pd.read_csv(f"./data/{user}_prob02_weather.csv")

#keep rows where value is tmax and tmin
df = df[df['ParameterName'].isin(['tmax', 'tmin'])]
df = df.pivot(index='Time', columns='ParameterName', values='ParameterValue') #make them columns
df['diff'] = df['tmax'] - df['tmin'] #difference of the two

# mean and standard definition
mean = df['diff'].mean()
std = df['diff'].std()

# count the number of values that are at most one standard deviation away from the mean and calculate the % of observations
count = ((df['diff'] >= mean - std) & (df['diff'] <= mean + std)).sum()
percentage = count / df.shape[0] * 100

print("a)\n\n Percentage of tmax observations at most one standard deviation away from the average: {:.2f}%".format(percentage))

# sort by temperature difference, descending
df.sort_values(by='diff', ascending=False, inplace=True)

#select the top-5 rows and print required info
df = df.head(5)
print("\nb)")
for i, row in df.iterrows():
    print("\nTimestamp: {}".format(i))
    print("Max temperature: {}".format(row['tmax']))
    print("Min temperature: {}".format(row['tmin']))
    print("Temperature difference: {}".format(row['diff']))

## Problem 3. Premier League Table
The file **private/exrc_02/data/AB0410_prob03_epl.csv** has some English Premier League results, downloaded using [this api](https://github.com/miquel-vv/football_data_api).

Using the full data, generate a league table which has the team name as the index and columns as follows (a win gives 3 points, a draw gives 1 point, and a loss gives 0 points):
* games played
* wins
* draws
* defeats
* goals for - goals against
* points


Sort it with points (most points win). If points are equal, then sorted by
* goal difference (goals for - goals against)
* goals for


The expected result should look something like this (not the same data though):
```
                games  wins  draws  defeats   goals  points
Man City           38    32      4        2  106-27     100
Man United         38    25      6        7   68-28      81
Tottenham          38    23      8        7   74-36      77
Liverpool          38    21     12        5   84-38      75
...
```

In [ ]:
import getpass as gt
import pandas as pd

user = gt.getuser().upper()
df = pd.read_csv(f"./data/{user}_prob03_epl.csv")

# extract the goals data from the fullTime column
df['goals'] = df['fullTime'].apply(lambda x: x.replace("{'homeTeam': ", '').replace(", 'awayTeam': ", '-').replace("}", ''))

# calculate the points
table['points'] = (table['wins'] * 3) + (table['draws'] * 1)

# sorting the league table
table.sort_values(by=['points', 'goals'], ascending=[False, False], inplace=True)
table.reset_index(inplace=True)
table = table[['homeTeam', 'games', 'wins', 'draws', 'defeats', 'goals', 'points']]

# INCOMPLETE


## Problem 4. Weather (part 2/2)

The file **private/exrc_02/data/AB0410_prob04_weather.txt** 
has some (old) weather data from Jyväskylä 1959-2021, again downloaded from [fmi.fi](https://en.ilmatieteenlaitos.fi/open-data-manual-fmi-wfs-services) (not recommended).

Calculate the "snow sum" (not a meteorological term) for each winter by adding the snow depths for each day of that winter. Start from winter 1959-60 and end to 2019-20 since 1958-59 and 2020-21 are only partial.

Notes:
* You need to define "winter" by yourself.
* FMI uses -1 as snow depth when "there is absolutely no snow at all". We don"t want to reduce snow sum in that case, so replace -1 with 0.
* For missing data, assume that the snow depth has been the same as during the previous day. (Fill NaNs with the previous valid value.)

Then produce a DataFrame that has the winter as the index (in form "1959-1960") and columns:
* snow sum
* snow sum rank among winters so that largest = 1
* count of days that snow depth has been over zero
* max snow depth of the winter.


The three first and the three last rows should look something like:
```
           Snow sum  rank  count  max
Winter                               
1959-1960      5593    18    169   65
1960-1961      5082    28    162   60
1961-1962      6644    12    156   78

...

2017-2018      6882     8    161   81
2018-2019      4030    42    150   54
2019-2020      1432    59    112   30
```

## Problem 5. Statfi data wrangle.
- Here we"re trying to make some sense out of the data that we downloaded from the statfi service in problem 5 of the first exercises. If you did it successfully, please use your own data. Otherwise you may use Harri"s adoption data at **public/statfi_adoptions_finnish.json**.
- Harri had the keyword *adopt*, and the final table *statfin_adopt_pxt_11lv.px*. You may want to replace them in the url below.
- First use [this front end](https://pxdata.stat.fi/PxWeb/pxweb/en/StatFin/StatFin__adopt/statfin_adopt_pxt_11lv.px/) to produce a meaningful table, and then try to produce a similar table with pandas and your data.
- The most elegant solution wins!

## Handin your final answers by running the code cell below.
- Save your latest changes first.
- Please remove anything that may identify you to your anonymous reviewer.
- You may run the code cell as many times as you wish.
- Your permission to write the handin file ends at the deadline.

In [ ]:
import sys
sys.path.append("/home/varpha/data_analytics/lib")
from handin import handin_exrc_02
handin_exrc_02()